# 从懂车帝获取二手车信息

## reference

[参考视频](https://www.bilibili.com/video/BV1RD421g7kL/?spm_id_from=333.337.search-card.all.click&vd_source=62be63bb97ad70a5484a980989eb1de5)

[参考博客1](https://cloud.tencent.com/developer/article/1831685)

[参考博客2](https://www.cnblogs.com/feng0815/p/16656656.html)

## keyword

- 字体反爬
- request
s
- fonttools



## href

[dongchedi_used_cars](https://www.dongchedi.com/usedcar/x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-1-1-x-x-x-x-x)



In [122]:
import requests
from fake_useragent import UserAgent
import json
import pandas as pd
from tqdm import tqdm
from time import sleep
# fake useragenet
ua = UserAgent()

user_agent = ua.random


# add headers 模拟浏览器正常访问
headers = {
    "User-Agent": user_agent,
    "Cookie": r"ttwid=1%7CmyHgF6b1xI5ibG3ITgfcq5ZVvIBbT6DE4VNW2h-JS_I%7C1721306154%7Ccead4d9da5b5fe6ae95fd319f1528c3b7199bb03307759ed95e0e3ce3812be28; tt_webid=7392953585342842430; tt_web_version=new; s_v_web_id=verify_lyr98o17_Df8Qunzk_DC92_4Rhh_B5MA_ptTG9CSea0WY; _gid=GA1.2.743883221.1721306153; city_name=%E9%87%8D%E5%BA%86; is_dev=false; is_boe=false; Hm_lvt_3e79ab9e4da287b5752d8048743b95e6=1721309659; HMACCOUNT=8160C1392AF61C82; Hm_lpvt_3e79ab9e4da287b5752d8048743b95e6=1721309665; msToken=-A6plihpMHPZ8KdBbgtAgwfzXN7BC68xK4EoeslAj10vvTlg-P6bDp7jDLu92wErp-fezviZiY9-45b_zD6QaTvYmWoMiFRZHXhPgXpl; _gat_gtag_UA_138671306_1=1; _ga=GA1.1.427450835.1721306153; _ga_YB3EWSDTGF=GS1.1.1721356560.2.1.1721356581.39.0.0",
}
# add data

data = [{
        "sh_city_name": "全国"
         , "page": int("{}".format(i))
         , "limit": 20
         } for i in range(1, 50 + 1, 1)]

# net
href = (
    r"https://www.dongchedi.com/motor/pc/sh/sh_sku_list?aid=1839&app_name=auto_web_pc"
)



# response
for _ in tqdm(range(len(data))):

    res = requests.post(href, headers=headers, data=data[_])

    # return status code
    res.raise_for_status

    # judge content encoding
    res.encoding = res.apparent_encoding
    # print(res.json())
    sleep(1)
    ## write middle
    with open ('./json/cars{}.json'.format(_+1), 'w', encoding = "utf-8") as f:
        json.dump(res.json(), f)


100%|██████████| 50/50 [01:45<00:00,  2.10s/it]


In [123]:
# replace
import re

wordMap = {
    r'\ue3eb':'7',
    r'\ue41d':'4',
    r'\ue439':'0',
    r'\ue45d':'9',
    r'\ue463':'2',
    r'\ue49d':'3',
    r'\ue4e3':'8',
    r'\ue534':'6',
    r'\ue54c':'1',
    r'\ue411':'5',
    r'\ue40a':'万'
}

def replace_(col):
    for key, value in wordMap.items():
        col = re.sub(key, value, col)
    return col

print(replace_('\ue463\ue54c.\ue45d\ue4e3\ue40a'))

print(replace_('\ue463.\ue534\ue411\ue40a'))

21.98万
2.65万


In [126]:
import os
# find all json files
json_files = []
for root, dirs, files in os.walk('.'):
    for file in files:
        if file.endswith('.json'):
            json_files.append(os.path.join(root, file))
json_files[:5]

['.\\json\\cars1.json',
 '.\\json\\cars10.json',
 '.\\json\\cars11.json',
 '.\\json\\cars12.json',
 '.\\json\\cars13.json',
 '.\\json\\cars14.json',
 '.\\json\\cars15.json',
 '.\\json\\cars16.json',
 '.\\json\\cars17.json',
 '.\\json\\cars18.json',
 '.\\json\\cars19.json',
 '.\\json\\cars2.json',
 '.\\json\\cars20.json',
 '.\\json\\cars21.json',
 '.\\json\\cars22.json',
 '.\\json\\cars23.json',
 '.\\json\\cars24.json',
 '.\\json\\cars25.json',
 '.\\json\\cars26.json',
 '.\\json\\cars27.json']

In [120]:
# read json file from html
import json
import pandas as pd
data = []
for jsonPath in json_files:
    with open(jsonPath, 'r', encoding='utf-8') as f_:
        car_data = json.load(f_)
        # header names
        Headers = [
            "brand_name"
            ,  "car_name"
            , "car_year"
            ,"official_price"
            , "sh_price"]

        data_searched = car_data['data']['search_sh_sku_info_list']

        # memory data

        # print(cars)

        for diclst in data_searched:

            # print(replace_(diclst['sh_price']))

            #还可以使用列表生成式
            # dit = [diclst[col]  if col not in 'official_price and sh_price' else replace_(diclst[col]) for col in Headers]
            dit = {col: diclst[col]for col in Headers}
            # print(dit)
            data.append(dit)

        cars = pd.DataFrame(columns=Headers, data=data)




In [121]:
cars

,brand_name,car_name,car_year,car_source_city_name,car_source_type,official_price,sh_price
0,比亚迪,1.5T 双冠旗舰Plus版,2015,,自营,.,.
1,比亚迪,秦EV300 尊享型,2017,沈阳,自营,.,.
2,吉利汽车,2.0L 自动尊贵型,2014,福州,自营,.,.
3,特斯拉,Performance高性能全轮驱动版,2019,庆,自营,.,.
4,路虎,2.0T 倾橙限量版,2015,郑州,自营,.,.
...,...,...,...,...,...,...,...
995,别克,15N CVT精英型,2020,连,自营,.,.
996,别克,典范 1.5L 自动精英型,2021,,自营,.,.
997,福特,EcoBoost 245 四驱豪翼型,2017,福州,自营,.,.
998,沃尔沃,S60L T4 智远版,2017,昆,自营,.,.


In [118]:
# you can also use dataframe-function apply
cars['sh_price'] = cars['sh_price'].apply(replace_)
cars['official_price'] = cars['official_price'].apply(replace_)




In [119]:
cars_duplicated = cars.drop_duplicates()
display(cars_duplicated)

,brand_name,car_name,car_year,official_price,sh_price
0,比亚迪,1.5T 双冠旗舰Plus版,2015,21.98万,2.65万
1,比亚迪,秦EV300 尊享型,2017,24.59万,3.68万
2,吉利汽车,2.0L 自动尊贵型,2014,11.99万,1.58万
3,特斯拉,Performance高性能全轮驱动版,2019,50.99万,15.99万
4,路虎,2.0T 倾橙限量版,2015,64.80万,7.98万
...,...,...,...,...,...
995,别克,15N CVT精英型,2020,9.39万,4.86万
996,别克,典范 1.5L 自动精英型,2021,12.59万,5.49万
997,福特,EcoBoost 245 四驱豪翼型,2017,24.58万,6.68万
998,沃尔沃,S60L T4 智远版,2017,30.99万,8.50万
